In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import sys

plt.rcParams["figure.figsize"] = [5, 3.1]

# Lecture 17: Quasi-Newton methods


In [ ]:
import feedback

feedback.feedback()

## Week 11 room changes!

| | | |
|-|-|-|
| Thu 14 Dec | 12:00-13:00 | Conference Auditorium 1 (GM.03) |
| Fri 15 Dec | 11:00-12:00 | Roger Stevens LT 21 |

## The problem

> Given a continuous function $f(x)$, the problem is to find a point $x^*$ such that $f(x^*) = 0$. That is, $x^*$ is a solution of the equation $f(x) = 0$ and is called a **root of $f(x)$**.

## Recall Newton's method:

$$x^{(i+1)} = x^{(i)} - \frac{f(x^{(i)})}{f'(x^{(i)})}$$

In [ ]:
def newton(f, df, x0, tol=1.0e-10):
    x = x0

    while abs(f(x)) > tol:
        x = x - f(x) / df(x)

    return x

Newton's method requires **function** and **it's derivative**.

This may not be possible since:

- $f(x)$ may be a "black box"
- the formula for $f(x)$ may be known but difficult to differentiate
- the derivative maybe *very* expensive to evaluate

# Approximating f'(x)

Let's approximate $f'(x)$ like we approximated $y'(t)$ when solving a differential equation!

In [ ]:
def f(x):
    return np.exp(-(x**2))


def plot(dt=1.0):
    t = np.linspace(-2, 2, 200)
    plt.plot(t, f(t))

    plt.plot([0, 0, -2], [0, f(0), f(0)], "--")
    plt.plot([dt, dt, -2], [0, f(dt), f(dt)], "--")

    slope = (f(dt) - f(0)) / dt
    plt.plot([0, dt], [f(0), f(dt)], label=f"{slope=:.1f}")

    plt.xticks([0, dt], ["t", "t + dt"])
    plt.yticks([f(0), f(dt)], ["f(t)", "f(t + dt)"])

    plt.grid("off")
    plt.legend()
    plt.show()


plot(0.5)

## Modified Newton's method

-   Recall that $f'(x) = \lim_{\mathrm{d}x \to 0} \frac{f(x + \mathrm{d}x) - f(x)}{\mathrm{d}x}$.

-   Hence we can choose a small value for $\mathrm{d}x$ (how small?) and approximate:

    $$
    f'(x) \approx \frac{f(x + \mathrm{d}x) - f(x)}{\mathrm{d}x}.
    $$

-   This modified-Newton method then becomes

    $$
    x^{(i+1)} = x^{(i)} - \frac{\mathrm{d}x \times f(x^{(i)})}{f(x^{(i)} + \mathrm{d}x) - f(x^{(i)})}.
    $$

In [ ]:
def modified_newton(f, x0, dx, tol=1.0e-10):
    x = x0

    while abs(f(x)) > tol:
        df = (f(x + dx) - f(x)) / dx  # two extra evaluations of f
        # per iteration
        x = x - f(x) / df

    return x

In [ ]:
def modified_newton2(f, x0, dx, tol=1.0e-10):
    x = x0
    fx = f(x)

    while abs(fx) > tol:
        df = (f(x + dx) - fx) / dx  # one extra evaluations of f
        # per iteration
        x = x - fx / df
        fx = f(x)  # one original evaluation of f per iteration

    return x

## Graphical representation of modified Newton

In [ ]:
x = np.linspace(-1, 1)
f = lambda x: np.tanh(x / 0.5 - 0.1)
df = lambda x: 2 * np.cosh(2.0 * x - 0.1) ** -2
fx = f(x)

plt.axhline(0, color="0")  # x = 0

x0 = 0.4
dx = 0.25

x1 = x0 - dx * f(x0) / (f(x0 + dx) - f(x0))

plt.plot([x0, x0, -1], [-1, f(x0), f(x0)], "0.80")
plt.plot([x0 + dx, x0 + dx, -1], [-1, f(x0 + dx), f(x0 + dx)], "0.80")
plt.plot([x1, x1], [-1, 0.0], "0.80")

plt.xticks([x0, x0 + dx, x1], ["$x^{(0)}$", "$x^{(0)} + \mathrm{d}x$", "$x^{(1)}$"])
plt.yticks([f(x0), f(x0 + dx)], ["$f(x^{(0)})$", "$f(x^{(0)} + \mathrm{d}x)$"])

plt.plot(x, fx)
l = plt.plot([x0, x0 + dx], [f(x0), f(x0) + dx], "o")
plt.plot(x1, f(x1), "o")

plt.plot(x, f(x0) + (x - x0) * (f(x0 + dx) - f(x0)) / dx, color=l[0].get_color())

plt.show()

## How to choose $\mathrm{d}x$?

**Smaller** - more accurate approximation

**Large** - too small and we will have rounding problems (subtracting two similar numbers)

## Example

-   When $f(x) = x^3$ then $f'(x) = 3 x^2$.

-   Hence, at $x_0 = 1$, $f(x_0) = 1$ and $f'(x_0) = 3$.

-   Consider what happens when we approximate this with python, using finite values for $\mathrm{d}x$.

In [ ]:
def cubic(x):
    return x**3


def d_cubic(x):
    return 3 * x**2


x = 1

headers = ["dx", "approx", "abs error", "rel error"]
data = []

for e in range(4, 18, 2):
    dx = 10**-e
    approx = (cubic(x + dx) - cubic(x)) / dx
    exact = d_cubic(x)
    abs_error = abs(exact - approx)
    rel_error = abs(exact - approx) / exact

    new_data = [dx, approx, abs_error, rel_error]
    data.append(new_data)

df = pd.DataFrame(data, columns=headers)
df.style.format(
    formatter={"dx": "{:e}", "approx": "{:f}", "abs error": "{:e}", "rel error": "{:e}"}
).hide_index().set_caption(
    "Simple approximation of a derivative using floating point arithmetic"
)

In [ ]:
xx = np.linspace(0.5, 1.5)
exact = d_cubic(xx)


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 6))
fig.suptitle("Approximations of the derivative")

ax1 = plt.subplot(121)
ax2 = plt.subplot(122)

for e in range(4, 18, 2):
    dx = 10**-e
    approx = (cubic(xx + dx) - cubic(xx)) / dx

    ax1.plot(xx, approx, label="dx=$10^{" + str(-e) + "}$")
    ax2.semilogy(xx, abs(approx - exact))

ax1.plot(xx, exact, "k--", label="exact")

ax1.set_xlabel("x")
ax1.set_ylabel("df")
ax1.grid()
ax1.legend()

ax2.set_xlabel("x")
ax2.set_ylabel("abs error")
ax2.grid("on")

plt.tight_layout()
plt.show()

## A special choice!

- Recall the definition of machine precision/unit roundoff from Lecture 3.
- The modified Newton method uses $\mathrm{d}x = \sqrt{eps}$.

In [ ]:
eps = np.finfo(np.double).eps
dx = np.sqrt(eps)

print(f"{eps=} {dx=}")

In [ ]:
xx = np.linspace(0.5, 1.5)
exact = d_cubic(xx)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 6))
fig.suptitle("A special approximation of the derivative")

for e in range(4, 18, 2):
    dx = 10**-e
    approx = (cubic(xx + dx) - cubic(xx)) / dx

    ax1.plot(xx, approx, "0.5")
    ax2.semilogy(xx, abs(approx - exact), "0.5")

ax1.plot(xx, exact, "k--")

eps = np.finfo(np.double).eps
dx = np.sqrt(eps)

approx = (cubic(xx + dx) - cubic(xx)) / dx

ax1.plot(xx, approx, "r", label=r"dx = $\sqrt{eps}$")
ax2.semilogy(xx, abs(approx - exact), "r", label=r"dx = $\sqrt{eps}$")

ax1.set_xlabel("x")
ax1.set_ylabel("df")
ax1.grid("on")
ax1.legend()

ax2.set_xlabel("x")
ax2.set_ylabel("abs error")
ax2.grid("on")

plt.tight_layout()
plt.show()

In [ ]:
def modified_newton_with_table(f, x0, tol=1.0e-10, maxiter=100):
    eps = np.finfo(np.double).eps
    dx = np.sqrt(eps)

    x = np.double(x0)
    it = 0

    headers = ["iter", "x", "f(x)"]
    data = []
    data.append([it, x, f(x)])

    while abs(f(x)) > tol:
        it += 1

        df = (f(x + dx) - f(x)) / dx
        x = x - f(x) / df
        data.append([it, x, f(x)])

        if it > maxiter:
            sys.stderr.write(
                "WARNING! Modified Newton iteration has not converged. "
                + "Too many iterations.\n"
            )
            break

    df = pd.DataFrame(data, columns=headers)
    return df.style.hide_index()

## Example

Recall the NACA0012 aerofoil example:

In [ ]:
def f_naca0012(x):
    t = 0.1

    yp = (
        -0.1015 * x**4
        + 0.2843 * x**3
        - 0.3516 * x**2
        - 0.126 * x
        + 0.2969 * np.sqrt(x)
    )
    f = yp - 0.5 * t

    return f

In [ ]:
def wing_shape(x):
    yp = (
        -0.1015 * np.power(x, 4)
        + 0.2843 * np.power(x, 3)
        - 0.3516 * np.power(x, 2)
        - 0.126 * x
        + 0.2969 * np.sqrt(x)
    )
    f = yp

    return f


t = np.linspace(0, 1, 1000)
p = plt.plot(t, wing_shape(t))
plt.plot(t, -wing_shape(t), color=p[0].get_color())
plt.axis("equal")
plt.grid("on")

plt.show()

Starting from $x^{(0)} = 1$ with $TOL = 10^{-4}$, we get the root as $x^* \approx 0.765789$ after 2 iterations for the NACA0012 aerofoil example (Same as Newton!):

In [ ]:
modified_newton_with_table(f_naca0012, x0=1.0, tol=1.0e-4)

Starting from $x^{(0)} = 0.1$ with $TOL = 10^{-4}$, we get the root as $x^* \approx 0.033863$ after 5 iterations for the second solution to the NACA0012 aerofoil example (Same as Newton!).

In [ ]:
modified_newton_with_table(f_naca0012, x0=0.1, tol=1.0e-4)

# The secant method

> Can we use a similar derivative approximation but avoid the extra function evaluation?

Recall modified-Newton method:

$$
x^{(i+1)} = x^{(i)} - \frac{\mathrm{d}x \times f(x^{(i)})}{f(x^{(i)} + \mathrm{d}x) - f(x^{(i)})}.
$$

Now with $\mathrm{d}x = x^{(i)} - x^{(i-1)}$ becomes

$$
x^{(i+1)} = x^{(i)} - \frac{(x^{(i)} - x^{(i-1)}) \times f(x^{(i)})}{f(x^{(i)}) - f(x^{(i-1)})}.
$$

In [ ]:
def secant(f, x0, x1, tol=1.0e-10):
    # two initial function evaluations
    f0 = f(x0)
    f1 = f(x1)

    while abs(f1) > tol:
        # compute derivative approximation
        df = (f1 - f0) / (x1 - x0)
        # update x
        x2 = x1 - f1 / df

        # update other variables
        x0, f0 = x1, f1
        x1, f1 = x2, f(x2)  # one evaluation of f per iteration

    return x1

## A geometric interpretation

In [ ]:
x = np.linspace(-1, 1)
f = lambda x: np.tanh(x / 0.5 - 0.1)
df = lambda x: 2 * np.cosh(2.0 * x - 0.1) ** -2
fx = f(x)

plt.axhline(0, color="0")  # x = 0

x0 = 0.4
x1 = 0.6
x2 = x1 - f(x1) * (x1 - x0) / (f(x1) - f(x0))

plt.plot([x0, x0, -1], [-1, f(x0), f(x0)], "0.80")
plt.plot([x1, x1, -1], [-1, f(x1), f(x1)], "0.80")
plt.plot([x2, x2], [-1, 0.0], "0.80")

plt.xticks([x0, x1, x2], ["$x^{(0)}$", "$x^{(1)}$", "$x^{(2)}$"])
plt.yticks([f(x0), f(x1)], ["$f(x^{(0)})$", "$f(x^{(1)})$"])

plt.plot(x, fx)
l = plt.plot([x0, x1], [f(x0), f(x1)], "o")
plt.plot(x2, f(x2), "o")

plt.plot(x, f(x0) + (x - x0) * (f(x1) - f(x0)) / (x1 - x0), color=l[0].get_color())

plt.show()

In [ ]:
def secant_with_table(f, x0, x1, tol=1.0e-10):
    f0 = f(x0)
    f1 = f(x1)

    it = 0

    headers = ["iter", "x0", "f(x0)", "x1", "f(x1)"]
    data = []
    data.append([it, x0, f0, x1, f1])

    while abs(f1) > tol:
        it = it + 1

        df = (f1 - f0) / (x1 - x0)
        x = x1 - f1 / df

        x0, f0 = x1, f1
        x1, f1 = x, f(x)  # one evaluation of f per iteration

        data.append([it, x0, f0, x1, f1])

    df = pd.DataFrame(data, columns=headers)
    return df.style.hide_index()

## Numerical examples

- The naca0012 example starting from 1 and 0.9 to a tolerance of $10^{-4}$ gives the solution as $x^* \approx 0.765264$ after 3 iterations (one more than Newton!)

In [ ]:
secant_with_table(f_naca0012, x0=1.0, x1=0.9, tol=1.0e-4)

Starting from $x^{(0)} = 0.0$ and $x^{(1)} = 0.1$ with $TOL = 10^{-4}$, we get the root as $x^* \approx 0.033870$ after 5 iterations for the second solution to the NACA0012 aerofoil example (Same as Newton!).

In [ ]:
secant_with_table(f_naca0012, x0=0.0, x1=0.1, tol=1.0e-4)

## Turning points?

In [ ]:
def f_turning(x):
    return (x - 1) ** 2


secant_with_table(f_turning, x0=4.0, x1=3.0, tol=1.0e-4)

## Other problems

Even more care is required to avoid divide by zero errors

In [ ]:
# equal function values at x0 and x1
secant(f_turning, x0=4.0, x1=-2.0, tol=1.0e-4)

In [ ]:
# too small tolerance means x0 = x1!
secant(f_turning, x0=4.0, x1=3.0, tol=1.0e-50)

## Exercise (hard!)

Find a periodic iteration for the secant method.

# Summary

|                    | Bisection         | Newton's method            | Modified Newton   | Secant            |
|--------------------|-------------------|----------------------------|-------------------|-------------------|
| Simple algorithm   | yes               | yes                        | yes               | yes               |
| Starting values    | bracket           | one                        | one               | two               |
| Iterations         | lots              | normally fewer             | similar to Newton | similar to Newton |
| Function evals     | one per iteration | `f` and `df` per iteration | two per iteration | one per iteration |
| Convergence        | with good bracket | not always                 | not always        | not always        |
| Turing point roots | no                | slower                     | slower            | slower            |
| Use of derivative  | no                | yes                        | no                | no                |

In [ ]:
def f(x):
    t = 0.1

    yp = (
        -0.1015 * np.power(x, 4)
        + 0.2843 * np.power(x, 3)
        - 0.3516 * np.power(x, 2)
        - 0.126 * x
        + 0.2969 * np.sqrt(x)
    )
    f = yp - 0.5 * t

    return f


def df(x):
    dy = (
        -4 * 0.1015 * np.power(x, 3)
        + 3 * 0.2843 * np.power(x, 2)
        - 2 * 0.3516 * x
        - 0.126
        + 0.2969 * 0.5 * np.power(x, -0.5)
    )
    f = dy

    return f


def bisection(func, a, b, tol=1.0e-10):
    # Starting values
    fa = func(a)
    func(b)

    feval = [2]
    rets = [(a + b) / 2]

    while b - a > tol:
        # Find new mid point
        c = (a + b) / 2
        fc = func(c)

        # if root is in left half of interval
        if fa * fc < 0.0:
            # move right end
            b = c
        else:
            # otherwise,
            # move the left end
            a = c
            fa = fc

        feval.append(feval[-1] + 1)
        rets.append((a + b) / 2)

    return rets, feval


def newton_values(f, df, x0, tol=1.0e-10):
    x = x0

    feval = [1]
    rets = [x]

    while abs(f(x)) > tol:
        x = x - f(x) / df(x)
        rets.append(x)
        feval.append(feval[-1] + 2)

    return rets, feval


def modified_newton_values(f, x0, tol=1.0e-10, maxiter=100):
    eps = np.finfo(np.double).eps
    dx = np.sqrt(eps)

    x = x0
    rets = [x]
    feval = [1]

    while abs(f(x)) > tol:
        df = (f(x + dx) - f(x)) / dx
        x = x - f(x) / df

        rets.append(x)
        feval.append(feval[-1] + 2)

    return rets, feval


def secant(f, x0, x1, tol=1.0e-10):
    # two initial function evaluations
    f0 = f(x0)
    f1 = f(x1)

    feval = [2]
    rets = [x1]

    while abs(f1) > tol:
        # compute derivative approximation
        df = (f1 - f0) / (x1 - x0)
        # update x
        x2 = x1 - f1 / df

        # update other variables
        x0, f0 = x1, f1
        x1, f1 = x2, f(x2)  # one evaluation of f per iteration

        rets.append(x1)
        feval.append(feval[-1] + 1)

    return rets, feval


tol = 1.0e-15
bisection_rets, bisection_fevals = bisection(f, 0.5, 1.0, tol=tol)
newton_rets, newton_fevals = newton_values(f, df, x0=1.0, tol=tol)
modified_rets, modified_fevals = modified_newton_values(f, x0=1.0, tol=tol)
secant_rets, secant_fevals = secant(f, x0=1.0, x1=0.5, tol=tol)

plt.figure(figsize=(8, 6))

plt.semilogy([abs(f(x)) for x in bisection_rets], ".-", label="bisection")
plt.semilogy([abs(f(x)) for x in newton_rets], ".-", label="Newton")
plt.semilogy([abs(f(x)) for x in modified_rets], ".-", label="Mod-Newton")
plt.semilogy([abs(f(x)) for x in secant_rets], ".-", label="secant")

plt.title("Rates of convergence for iterations")
plt.xlabel("iter ($i$)")
plt.ylabel("$|f(x^{(i)})|$")
plt.legend()
plt.grid("on")
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))

plt.semilogy(
    bisection_fevals, [abs(f(x)) for x in bisection_rets], ".-", label="bisection"
)
plt.semilogy(newton_fevals, [abs(f(x)) for x in newton_rets], ".-", label="Newton")
plt.semilogy(
    modified_fevals, [abs(f(x)) for x in modified_rets], ".-", label="Mod-Newton"
)
plt.semilogy(secant_fevals, [abs(f(x)) for x in secant_rets], ".-", label="secant")

plt.title("Rates of convergence for computational cost")
plt.xlabel("function (or derivative) evaluations")
plt.ylabel("$|f(x^{(i)})|$")
plt.legend()
plt.grid("on")
plt.show()